In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('/content/dataset.csv')

In [ ]:
data.head()

,natural_language,terminal_command
0,List all files in the current directory,ls
1,Show me all the files in this directory,ls
2,Can you display the files in the current folder?,ls
3,Please list everything in this directory,ls
4,Show all files in the current directory,ls


In [ ]:
train_nl= data["natural_language"].tolist()
train_bash=data["terminal_command"].tolist()

In [ ]:
train_nl[0],train_bash[0]

('List all files in the current directory', 'ls')

In [ ]:
inputs=["translate to bash : "+nl for nl in train_nl]

In [ ]:
inputs[:2]

['translate to bash : List all files in the current directory',
 'translate to bash : Show me all the files in this directory']

In [ ]:
targets=train_bash

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import Trainer,TrainingArguments,AutoModelForCausalLM

In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"



In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch
from peft import LoraConfig,get_peft_model

In [ ]:
train_data,eval_data=train_test_split(data,test_size=0.1,random_state=42)


 converting to huggingface data format

In [ ]:
train_dataset=Dataset.from_pandas(train_data)
eval_dataset=Dataset.from_pandas(eval_data)

tokenizing dataset


In [ ]:
def preprocess_function(text):
  model_inputs=tokenizer(text["natural_language"],padding="max_length",truncation=True)
  labels=tokenizer(text["terminal_command"],padding="max_length",truncation=True)
  model_inputs["labels"]=labels["input_ids"]
  return model_inputs

In [ ]:
# tokenization

train_dataset=train_dataset.map(preprocess_function,batched=True)

eval_dataset=eval_dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [ ]:
# to improve precision

In [ ]:
# !pip uninstall -y bitsandbytes
# !pip install -U bitsandbytes


In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig (
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16, # to improve precision
    bnb_4bit_use_double_quant = True # Double quantization reduces memory usage
)


In [ ]:
# Load the model

model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", quantization_config = bnb_config, device_map="auto")

In [ ]:
lora_config=LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"



)

In [ ]:
model=get_peft_model(model,lora_config)


In [ ]:
from accelerate import infer_auto_device_map

In [ ]:
device_map = infer_auto_device_map(model, max_memory = {0: "12GiB", "cpu": "2GiB"})


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    fp16=True,
    logging_dir="./results/logs",
    logging_steps=10,
    save_steps=500,
    do_eval=True,
    eval_steps=500,
    save_total_limit=2
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset


)

In [ ]:
trainer.train()

Step,Training Loss
10,5.758200
20,3.817800
30,1.918300
40,0.678400
50,0.215800
60,0.101600
70,0.076300
80,0.084800
90,0.077600
100,0.077800


In [ ]:

model.save_pretrained('/content/saved_model')
tokenizer.save_pretrained('/content/saved_model')

('/content/saved_model/tokenizer_config.json',
 '/content/saved_model/special_tokens_map.json',
 '/content/saved_model/chat_template.jinja',
 '/content/saved_model/tokenizer.model',
 '/content/saved_model/added_tokens.json',
 '/content/saved_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load saved model & tokenizer
model_path = "/content/saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Encode prompt
prompt = input("> ")
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(**inputs, max_length=50)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)


> Please list everything in this directory	
Please list everything in this directory	


### NEED A LOT MORE EPOCHS TO BE TRAINED

In [ ]:
!zip -r saved_model.zip /content/saved_model
from google.colab import files
files.download("saved_model.zip")


updating: content/saved_model/ (stored 0%)
updating: content/saved_model/special_tokens_map.json (deflated 79%)
updating: content/saved_model/README.md (deflated 66%)
updating: content/saved_model/tokenizer_config.json (deflated 69%)
updating: content/saved_model/tokenizer.json (deflated 85%)
updating: content/saved_model/tokenizer.model (deflated 55%)
updating: content/saved_model/adapter_config.json (deflated 56%)
updating: content/saved_model/adapter_model.safetensors (deflated 8%)
updating: content/saved_model/chat_template.jinja (deflated 60%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("saved_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>